In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

import pandas as pd, numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error


In [ ]:
data_train = pd.read_csv("/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/train.csv")
data_test = pd.read_csv("/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/test.csv")
data_submission = pd.read_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/sample_submission.csv')

In [ ]:
def add_features(df):
    df['cross']= df['u_in'] * df['u_out']
    df['cross2']= df['time_step'] * df['u_out']
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    print("Step-1...Completed")
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    print("Step-2...Completed")
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_in__mean'] = df.groupby(['breath_id'])['u_in'].transform('mean')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

    print("Step-3...Completed")
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    print("Step-4...Completed")
    
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] /df['count']
    
    df['breath_id_lag']=df['breath_id'].shift(1).fillna(0)
    df['breath_id_lag2']=df['breath_id'].shift(2).fillna(0)
    df['breath_id_lagsame']=np.select([df['breath_id_lag']==df['breath_id']],[1],0)
    df['breath_id_lag2same']=np.select([df['breath_id_lag2']==df['breath_id']],[1],0)
    df['breath_id__u_in_lag'] = df['u_in'].shift(1).fillna(0)
    df['breath_id__u_in_lag'] = df['breath_id__u_in_lag'] * df['breath_id_lagsame']
    df['breath_id__u_in_lag2'] = df['u_in'].shift(2).fillna(0)
    df['breath_id__u_in_lag2'] = df['breath_id__u_in_lag2'] * df['breath_id_lag2same']
    print("Step-5...Completed")
    
    df['time_step_diff'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['ewm_u_in_mean'] = (df\
                           .groupby('breath_id')['u_in']\
                           .ewm(halflife=9)\
                           .mean()\
                           .reset_index(level=0,drop=True))
    df[["15_in_sum","15_in_min","15_in_max","15_in_mean"]] = (df\
                                                              .groupby('breath_id')['u_in']\
                                                              .rolling(window=15,min_periods=1)\
                                                              .agg({"15_in_sum":"sum",
                                                                    "15_in_min":"min",
                                                                    "15_in_max":"max",
                                                                    "15_in_mean":"mean"
                                                                    #"15_in_std":"std"
                                                               })\
                                                               .reset_index(level=0,drop=True))
    print("Step-6...Completed")
        
    df['u_in_lagback_diff1'] = df['u_in'] - df['u_in_lag_back1']
    df['u_out_lagback_diff1'] = df['u_out'] - df['u_out_lag_back1']
    df['u_in_lagback_diff2'] = df['u_in'] - df['u_in_lag_back2']
    df['u_out_lagback_diff2'] = df['u_out'] - df['u_out_lag_back2']
    print("Step-7...Completed")
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    print("Step-8...Completed")
    
    return df

train = add_features(data_train)
test = add_features(data_test)

Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed
Step-1...Completed
Step-2...Completed
Step-3...Completed
Step-4...Completed
Step-5...Completed
Step-6...Completed
Step-7...Completed
Step-8...Completed


In [ ]:
print('Train shape is now:', train.shape )
train.head()

Train shape is now: (6036000, 73)


,id,breath_id,time_step,u_in,u_out,pressure,cross,cross2,area,time_step_cumsum,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,1,0.000000,0.083334,0,5.837492,0.0,0.0,0.000000,0.000000,...,1,0,0,1,0,0,0,0,0,0
1,2,1,0.033652,18.383041,0,5.907794,0.0,0.0,0.618632,0.033652,...,1,0,0,1,0,0,0,0,0,0
2,3,1,0.067514,22.509278,0,7.876254,0.0,0.0,2.138333,0.101167,...,1,0,0,1,0,0,0,0,0,0
3,4,1,0.101542,22.808822,0,11.742872,0.0,0.0,4.454391,0.202709,...,1,0,0,1,0,0,0,0,0,0
4,5,1,0.135756,25.355850,0,12.234987,0.0,0.0,7.896588,0.338464,...,1,0,0,1,0,0,0,0,0,0


Auxiliary Target

In [ ]:
train['pressure_diff'] = train.groupby('breath_id').pressure.diff().fillna(0)
train['pressure_integral'] = train.groupby('breath_id').pressure.cumsum()/200
targets = train[['pressure','pressure_diff','pressure_integral']].to_numpy().reshape(-1, 80, 3)

train.drop(['pressure','pressure_diff','pressure_integral','id', 'breath_id','one','count',
            'breath_id_lag','breath_id_lag2','breath_id_lagsame',
            'breath_id_lag2same'], axis=1, inplace=True)

test.drop(['id', 'breath_id','one','count','breath_id_lag',
            'breath_id_lag2','breath_id_lagsame',
            'breath_id_lag2same'], axis=1, inplace=True)

In [ ]:
print('Targets shape is',targets.shape)

Targets shape is (75450, 80, 3)


Rearrange Column Order

In [ ]:
COL_ORDER = list(train.columns[:3]) + list(train.columns[-15:]) + list(train.columns[3:-15])
train = train[COL_ORDER]
test = test[COL_ORDER]

print('Train columns:')
np.array( COL_ORDER )

Train columns:


array(['time_step', 'u_in', 'u_out', 'R_20', 'R_5', 'R_50', 'C_10',
       'C_20', 'C_50', 'R__C_20__10', 'R__C_20__20', 'R__C_20__50',
       'R__C_50__10', 'R__C_50__20', 'R__C_50__50', 'R__C_5__10',
       'R__C_5__20', 'R__C_5__50', 'cross', 'cross2', 'area',
       'time_step_cumsum', 'u_in_cumsum', 'u_in_lag1', 'u_out_lag1',
       'u_in_lag_back1', 'u_out_lag_back1', 'u_in_lag2', 'u_out_lag2',
       'u_in_lag_back2', 'u_out_lag_back2', 'u_in_lag3', 'u_out_lag3',
       'u_in_lag_back3', 'u_out_lag_back3', 'u_in_lag4', 'u_out_lag4',
       'u_in_lag_back4', 'u_out_lag_back4', 'breath_id__u_in__max',
       'breath_id__u_in__mean', 'breath_id__u_in__diffmax',
       'breath_id__u_in__diffmean', 'u_in_diff1', 'u_out_diff1',
       'u_in_diff2', 'u_out_diff2', 'u_in_diff3', 'u_out_diff3',
       'u_in_diff4', 'u_out_diff4', 'u_in_cummean', 'breath_id__u_in_lag',
       'breath_id__u_in_lag2', 'time_step_diff', 'ewm_u_in_mean',
       '15_in_sum', '15_in_min', '15_in_max', '15_in_me

Normalize Features

In [ ]:
RS = RobustScaler()
train = RS.fit_transform(train.astype('float32'))
test = RS.transform(test.astype('float32'))

train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

Mask Loss

In [ ]:
U_OUT_IDX = 2
y_weight = np.ones_like( targets )
u_out_values = train[:,:,U_OUT_IDX]
y_weight[ u_out_values==0 ] = 0 # because robust scaler changes 1 to 0

In [ ]:
train.shape, targets.shape, y_weight.shape

((75450, 80, 64), (75450, 80, 3), (75450, 80, 3))

Build model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,GRU,SimpleRNN, RNN, Input, Bidirectional,LayerNormalization,BatchNormalization
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import mean_squared_error 

In [ ]:
# n=5
# cross_method = KFold(n_splits=n, shuffle=True)

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# def model_fit(models, epochs = 50,batch_size = 64):
#     t=0 #KFlod times
#     mse = np.zeros(len(models))
#     mae = np.zeros(len(models))
#     # for train_index, test_index in cross_method.split(train, targets):
#     for train_index, test_index in cross_method(train, targets,test_size=0.2):
#         X_train, X_valid = train[train_index], train[test_index]
#         y_train, y_valid = targets[train_index], targets[test_index]
#         for id , model in enumerate(models):
#             model.fit(X_train, y_train , epochs = epochs, batch_size = batch_size ,validation_data = (X_valid, y_valid))
#             y_pred = model.predict(X_valid)
#             for i in range(len(y_pred)):
#                 mse[id] += mean_squared_error(y_valid[i], y_pred[i]) / n
#                 mae[id] += mean_absolute_error(y_valid[i], y_pred[i]) / n

#         t+=1
#         print(t,"/10")
#     mse = mse/len(y_pred)
#     mae = mae/len(y_pred)
    
#     return(mse, mae)

In [ ]:
from sklearn.model_selection import train_test_split
import os
# cross_method = train_test_split\
model_dir = 'lab2-logs/model/'
# os.makedirs(model_dir)  
log_dir = os.path.join('lab2-logs', 'model')
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
checkpoint_filepath = "Best_model.hdf5"
# sv = keras.callbacks.ModelCheckpoint(
#         checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
#         save_weights_only=True,
#         mode='auto', save_freq='epoch',
#         options=None)
tb = keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)
rlrop = keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.1,patience=4)


In [ ]:
NUM_FOLDS = 5
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
test_preds = []
for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
  print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
  X_train, X_valid = train[train_idx], train[test_idx]
  y_train, y_valid = targets[train_idx], targets[test_idx]

--------------- > Fold 1 < ---------------
--------------- > Fold 2 < ---------------
--------------- > Fold 3 < ---------------
--------------- > Fold 4 < ---------------
--------------- > Fold 5 < ---------------


In [ ]:

def model_fit(models, epoch = 50,batch_size = 64,path = checkpoint_filepath):
    
    
    t=0 #KFlod times
    predict_result = []
    mse = np.zeros(len(models))
    mae = np.zeros(len(models))
    # for train_index, test_index in cross_method.split(train, targets):
    # for train_index, test_index in cross_method(train, targets,test_size=0.2):
    # X_train, X_valid,y_train, y_valid = train_test_split(train,targets,test_size=0.2)
    for id , model in enumerate(models):
        sv = keras.callbacks.ModelCheckpoint(
        path, monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True,
        mode='auto', save_freq='epoch',
        options=None)
    
        model.fit(X_train, y_train , epochs = epoch, batch_size = batch_size ,validation_data = (X_valid, y_valid ,y_weight[test_idx,:,:1]),callbacks=[sv, tb, rlrop],sample_weight=y_weight[train_idx,:,:1])
        y_pred = model.predict(X_valid)
        for i in range(len(y_pred)):
            mse[id] += mean_squared_error(y_valid[i], y_pred[i]) 
            mae[id] += mean_absolute_error(y_valid[i], y_pred[i])
        predict_result.append(model.predict(test))
        # t+=1
        # print(t,"/10")
    mse = mse/len(y_pred)
    mae = mae/len(y_pred)
    

    return(mse, mae,predict_result)

In [ ]:
def create_LSTM(out_dim=3):    ##build and compile LSTM model
    
    model=Sequential()
    model.add(LSTM(128,input_shape= [None,64], return_sequences = True))
    model.add(LSTM(64,input_shape= [None,64], return_sequences =  True))
    model.add(Dense(out_dim))
    
    model.compile(loss='mean_absolute_error',optimizer=opt, weighted_metrics = keras.losses.MeanAbsoluteError())
    return model
    
model_LSTM = create_LSTM()
model_LSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 128)         98816     
                                                                 
 lstm_1 (LSTM)               (None, None, 64)          49408     
                                                                 
 dense (Dense)               (None, None, 3)           195       
                                                                 
Total params: 148,419
Trainable params: 148,419
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def create_GRU(out_dim=3):    ##build and compile GRU model
    
    model = Sequential()
    model.add(GRU(128,input_shape= [None,64], return_sequences = True))
    model.add(GRU(64,input_shape= [None,64],return_sequences = True))
    model.add(Dense(out_dim))
    
    model.compile(loss='mean_absolute_error',optimizer=opt, weighted_metrics = keras.losses.MeanAbsoluteError())
    return model
    
model_GRU = create_GRU()
model_GRU.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 128)         74496     
                                                                 
 gru_1 (GRU)                 (None, None, 64)          37248     
                                                                 
 dense_1 (Dense)             (None, None, 3)           195       
                                                                 
Total params: 111,939
Trainable params: 111,939
Non-trainable params: 0
_________________________________________________________________


In [ ]:
models = [model_LSTM,model_GRU]
result = model_fit(models,epoch=1,path= 'model_basic.hdf5')
print("LSTM :")
print(f"    MSE = {result[0][0]:.5f}, MAE = {result[0][1]:.5f}")  
print("GRU :")
print(f"    MSE = {result[1][0]:.5f}, MAE = {result[1][1]:.5f}") 
model_predict = pd.DataFrame(np.array(result[2]).reshape([-1,3]),columns = ['pressure','pressure_diff','pressure_cumsum'])
model_predict.index +=1
model_predict[:4024000]['pressure'].to_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/basic_results-lstm.csv',index_label='id')
model_predict_gru = model_predict[4024000:]['pressure'].reset_index()
model_predict_gru.index += 1
model_predict_gru['pressure'].to_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/basic_results-gru.csv',index_label = 'id')


944/944 [==============================] - ETA: 0s - loss: 0.1363 - mean_absolute_error: 3.4697
Epoch 1: val_loss improved from inf to 0.13995, saving model to model_basic.hdf5
1572/1572 [==============================] - 76s 48ms/step


In [ ]:
# model_predict = pd.DataFrame(np.array(result[2]).reshape([-1,3]),columns = ['pressure','pressure_diff','pressure_cumsum'])
# model_predict


,pressure,pressure_diff,pressure_cumsum
0,5.686353,-0.003121,0.036551
1,5.642247,-0.375608,0.081033
2,6.743396,0.901900,0.094866
3,7.828964,1.017391,0.115886
4,9.112551,1.253094,0.171885
...,...,...,...
8047995,35.730602,0.596696,4.048653
8047996,36.077049,0.464973,4.066438
8047997,36.424080,0.510211,4.065267
8047998,37.263638,0.377469,4.007729


Add BatchNormalization

Build BN model

In [ ]:
def create_LSTM_BN(out_dim=3):    ##build and compile LSTM model
    
    model=Sequential()
    model.add(LSTM(128,input_shape= [None,64], return_sequences = True))
    model.add(BatchNormalization())
    model.add(LSTM(64,input_shape= [None,64], return_sequences =  True))
    model.add(Dense(out_dim,activation="linear"))
    
    model.compile(loss='mean_absolute_error',optimizer=opt)
    return model
    
model_LSTM_BN = create_LSTM_BN()
model_LSTM_BN.summary()

In [ ]:
def create_GRU_BN(out_dim=3):    ##build and compile GRU model
    
    model = Sequential()
    model.add(GRU(128,input_shape= [None,64], return_sequences = True))
    model.add(BatchNormalization())
    model.add(GRU(64,input_shape= [None,64],return_sequences = True))
    model.add(Dense(out_dim))
    
    model.compile(loss='mean_absolute_error',optimizer=opt)
    return model
    
model_GRU_BN = create_GRU_BN()
model_GRU_BN.summary()

In [ ]:
models = [model_LSTM_BN,model_GRU_BN]
result = model_fit(models,path= 'model_BN.hdf5')
print("LSTM_BN :")
print(f"    MSE = {result[0][0]:.5f}, MAE = {result[0][1]:.5f}")  
print("GRU_BN :")
print(f"    MSE = {result[1][0]:.5f}, MAE = {result[1][1]:.5f}") 
model_predict = pd.DataFrame(np.array(result[2]).reshape([-1,3]),columns = ['pressure','pressure_diff','pressure_cumsum'])
model_predict.index +=1
model_predict[:4024000]['pressure'].to_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/BN_results-lstm.csv',index_label='id')
model_predict_gru = model_predict[4024000:]['pressure'].reset_index()
model_predict_gru.index += 1
model_predict_gru['pressure'].to_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/BN_results-gru.csv',index_label='id')


In [ ]:
def create_LSTM_LN(out_dim=3):    ##build and compile LSTM model
    
    model=Sequential()
    model.add(LSTM(128,input_shape= [None,64], return_sequences = True))
    model.add(LayerNormalization())
    model.add(LSTM(64,input_shape= [None,64], return_sequences =  True))
    model.add(Dense(out_dim,activation="linear"))
    
    model.compile(loss='mean_absolute_error',optimizer=opt)
    return model
    
model_LSTM_LN = create_LSTM_LN()
model_LSTM_LN.summary()

In [ ]:
def create_GRU_LN(out_dim=3):    ##build and compile GRU model
    
    model = Sequential()
    model.add(GRU(128,input_shape= [None,64], return_sequences = True))
    model.add(LayerNormalization())
    model.add(GRU(64,input_shape= [None,64],return_sequences = True))
    model.add(Dense(out_dim))
    
    model.compile(loss='mean_absolute_error',optimizer=opt)
    return model
    
model_GRU_LN = create_GRU_LN()
model_GRU_LN.summary()

In [ ]:
LN_models = [model_LSTM_LN,model_GRU_LN]
result = model_fit(LN_models,path= 'model_LN.hdf5')
print("LSTM_LN :")
print(f"    MSE = {result[0][0]:.5f}, MAE = {result[0][1]:.5f}")  
print("GRU_LN :")
print(f"    MSE = {result[1][0]:.5f}, MAE = {result[1][1]:.5f}") 

In [ ]:
model_predict = pd.DataFrame(np.array(result[2]).reshape([-1,3]),columns = ['pressure','pressure_diff','pressure_cumsum'])
model_predict.index +=1
model_predict[:4024000]['pressure'].to_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/LN_results-lstm.csv',index_label="id")
model_predict_gru = model_predict[4024000:]['pressure'].reset_index()
model_predict_gru.index += 1
model_predict_gru['pressure'].to_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/LN_results-gru.csv',index_label="id")

Build BiLSTM model

In [ ]:
def create_biLSTM(out_dim=3):    ##build and compile LSTM model
    
    input = Input(shape=[None,64])

    layer_BiLSTM1 = Bidirectional(LSTM(128,input_shape= [None,64], return_sequences = True))(input)
    layer_BiLSTM2 = Bidirectional(LSTM(128,input_shape= [None,64], return_sequences = True))(layer_BiLSTM1)
    layer_output = Dense(out_dim,activation="linear")(layer_BiLSTM2)
    
    model = keras.models.Model(inputs = input,outputs = layer_output)

    model.compile(loss='mean_absolute_error',optimizer=opt)
    return model
    
model_biLSTM = create_biLSTM()
model_biLSTM.summary()

In [ ]:
model = [model_biLSTM]
result = model_fit(model,path='model_biLSTM.hdf5')
print("LSTM_Bi :")
print(f"    MSE = {float(result[0]):.5f}, MAE = {float(result[1]):.5f}")   

In [ ]:
model_predict = pd.DataFrame(np.array(result[2]).reshape([-1,3]),columns = ['pressure','pressure_diff','pressure_cumsum'])
model_predict.index +=1
model_predict['pressure'].to_csv('/content/drive/MyDrive/數據科學方法/Computer Project/ventilator-pressure-prediction/BiLSTM_results.csv',index_label='id')
